In [ ]:
!pip install torch
!pip install tensorflow
!pip install simpletransformers
!pip install datasets
!nvidia-smi
!pip install transformers[torch]
!pip install accelerate==0.27.2 -U
!pip install datasets

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import torch
import tensorflow as tf
import pandas as pd
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import string
import re

In [ ]:
torch.cuda.is_available()

True

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
class DataPreprocessor():
    def __init__(self):
        pass
    def handle_columns(self, dataframe, *columns):   # sütun isimlerini günceller eski isimleri kaldırır.
      i=0;
      for num in range(0, len(columns)+1):
        column_name = dataframe.keys().tolist()[i]
        dataframe.rename({column_name: columns[0][i]}, axis=1, inplace=True)
        i = i + 1
      return dataframe
    def preprocess(self, dataframe):                 # texti taglerden, uzun boşluklardan ve noktalama işaretlerinden arındırır.
      i=0
      X = dataframe["text"]
      for i in range(0, len(X)-1):
        X[i] = X[i].lower()
        X[i] = re.sub(r'<.*?>', '', X[i])
        X[i] = re.sub(r'\s+', ' ', X[i])
        for char in X[i]:
          ascii_val=ord(char)
          if 33 <= ascii_val <= 64:
                X[i] = X[i].replace(char, '')
        i = i + 1
      return X
    def handle_missing_values(self, dataframe):      # eğer boş veriler label ya da text varsa o satırları siler
        dataframe.dropna(inplace=True)
        return dataframe

In [ ]:
from datasets import load_dataset
import pandas as pd

# Veri setini yükle
dataset = load_dataset("batubayk/TR-News")

# Dataset'i pandas DataFrame'e dönüştür
df = pd.DataFrame(dataset['train'])

# İlgili sütunları seç ve NaN değerleri çıkar
df = df[["content", "topic"]].dropna()

print(df.head())

Generating train split:   0%|          | 0/277573 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/14610 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/15379 [00:00<?, ? examples/s]

                                             content    topic
0  Tuğba Özerk, “Duymayan Kalmasın” programında, ...   Fiskos
1  MHP Genel Başkanı Bahçeli, Manisa'nın Selendi ...  Türkiye
2  Rusya’nın Suriye’ye havadan müdahaleye başlama...    dunya
3  Galatasaray’da forvet transferinde halen somut...   futbol
4  İspanya'nın Marca Gazetesi'nde yayımlanan deme...     Spor


In [ ]:
df.loc[df["topic"].isin(df["topic"].value_counts()[df["topic"].value_counts() < 1000].index),"topic"] = "Diğer"
df.loc[df["topic"]=="spor", "topic"] = "Spor"
df.loc[df["topic"]=="dunya", "topic"] = "Dünya"
df.loc[df["topic"]=="bilim_ve_teknoloji", "topic"] = "Teknoloji"
df.loc[df["topic"]=="futbol", "topic"] = "Spor"
df.loc[df["topic"]=="turkiye", "topic"] = "Türkiye"
df.loc[df["topic"]=="ekonomi", "topic"] = "Ekonomi"
df.loc[df["topic"]=="Fiskos", "topic"] = "Diğer"
df.loc[df["topic"]=="siyaset", "topic"] = "Siyaset"
df.loc[df["topic"]=="Sanat", "topic"] = "Kültür-Sanat"
df.loc[df["topic"]=="Futbol", "topic"] = "Spor"
df.loc[df["topic"]=="Yaşam", "topic"] = "İş-Yaşam"
df.loc[df["topic"]=="Türkiye", "topic"] = "Gündem"
df.loc[df["topic"]=="Dünya", "topic"] = "Gündem"
df.loc[df["topic"]=="Siyaset", "topic"] = "Gündem"
df.loc[df["topic"]=="Sağlık", "topic"] = "Diğer"
df.loc[df["topic"]=="Otomobil", "topic"] = "Diğer"
df.loc[df["topic"]=="Eğitim", "topic"] = "Diğer"


In [ ]:
def sample_extractor(dataframe, *topics):
    # Take an initial sample from the dataframe
    old_topic = dataframe.sample(1)
    topics = topics[0]
    for topic in topics:
        if topic in dataframe['topic'].values:
            # Check if there are at least 500 rows for the given topic
            new_topic = dataframe[dataframe["topic"] == topic]
            if len(new_topic) >= 2500:
                new_topic = new_topic.sample(2500)
            else:
                new_topic = new_topic.sample(len(new_topic))  # Sample all available rows if fewer than 500
            old_topic = pd.concat([old_topic, new_topic], axis=0)

    return old_topic


In [ ]:
new_df = sample_extractor(df, ["Spor", "Teknoloji", "Ekonomi", "Kültür-Sanat"])
new_df.head()

,content,topic
228478,"Başbakan Tayyip Erdoğan'ın ""İsrail'e tamir içi...",Gündem
263134,Süper Lig'de 19. sezonunda 3. kez lige veda ed...,Spor
27605,İspanya La Liga’nın son haftasında Real Madrid...,Spor
50231,"İstanbul'da Kasımpaşa, sahasında Fenerbahçe'yi...",Spor
94365,UEFA Avrupa Ligi 3. Ön Eleme Turu ilk maçında ...,Spor


In [ ]:
print(new_df["topic"].value_counts())

topic
Spor            2500
Ekonomi         2500
Teknoloji       2500
Kültür-Sanat    2500
Gündem             1
Name: count, dtype: int64


In [ ]:
df = new_df.reset_index().drop(labels=["index"], axis=1)
df = df[1:]
df = df.reset_index().drop(labels=["index"], axis=1)
df.head()

,content,topic
0,Süper Lig'de 19. sezonunda 3. kez lige veda ed...,Spor
1,İspanya La Liga’nın son haftasında Real Madrid...,Spor
2,"İstanbul'da Kasımpaşa, sahasında Fenerbahçe'yi...",Spor
3,UEFA Avrupa Ligi 3. Ön Eleme Turu ilk maçında ...,Spor
4,Galatasaray'ın yıldızı Wesley Sneijder'den Fen...,Spor


In [ ]:
import re
data_handler = DataPreprocessor()
print(df.head())
df = data_handler.handle_columns(df, ["text", "label"])
print(df.head())
df["text"] = data_handler.preprocess(df)
df = data_handler.handle_missing_values(df)
print(df.head())

                                             content topic
0  Süper Lig'de 19. sezonunda 3. kez lige veda ed...  Spor
1  İspanya La Liga’nın son haftasında Real Madrid...  Spor
2  İstanbul'da Kasımpaşa, sahasında Fenerbahçe'yi...  Spor
3  UEFA Avrupa Ligi 3. Ön Eleme Turu ilk maçında ...  Spor
4  Galatasaray'ın yıldızı Wesley Sneijder'den Fen...  Spor
                                                text label
0  Süper Lig'de 19. sezonunda 3. kez lige veda ed...  Spor
1  İspanya La Liga’nın son haftasında Real Madrid...  Spor
2  İstanbul'da Kasımpaşa, sahasında Fenerbahçe'yi...  Spor
3  UEFA Avrupa Ligi 3. Ön Eleme Turu ilk maçında ...  Spor
4  Galatasaray'ın yıldızı Wesley Sneijder'den Fen...  Spor
                                                text label
0  süper ligde  sezonunda  kez lige veda eden den...  Spor
1  i̇spanya la liga’nın son haftasında real madri...  Spor
2  i̇stanbulda kasımpaşa sahasında fenerbahçeyi a...  Spor
3  uefa avrupa ligi  ön eleme turu ilk maçında st...  Sp

In [ ]:
id = [i for i in range(0, len(df))]
label = df["label"].unique().tolist()

In [ ]:
labels = df.label.unique().tolist()
labels = [s.strip() for s in labels]
print(labels)

['Spor', 'Teknoloji', 'Ekonomi', 'Kültür-Sanat']


In [ ]:
id2label = {0: "Spor", 1: "Teknoloji", 2: "Ekonomi", 3: "Kültür-Sanat"}
label2id = {label:id for id,label in enumerate(labels)}

In [ ]:
id2label

{0: 'Spor', 1: 'Teknoloji', 2: 'Ekonomi', 3: 'Kültür-Sanat'}

In [ ]:
label2id

{'Spor': 0, 'Teknoloji': 1, 'Ekonomi': 2, 'Kültür-Sanat': 3}

In [ ]:
df['labels']=df.label.map(lambda x: label2id[x.strip()])

In [ ]:
df.head()

,text,label,labels
0,süper ligde sezonunda kez lige veda eden den...,Spor,0
1,i̇spanya la liga’nın son haftasında real madri...,Spor,0
2,i̇stanbulda kasımpaşa sahasında fenerbahçeyi a...,Spor,0
3,uefa avrupa ligi ön eleme turu ilk maçında st...,Spor,0
4,galatasarayın yıldızı wesley sneijderden fener...,Spor,0


In [ ]:
df.sample(10)

,text,label,labels
7384,altının gram fiyatı bugün de dolar kurundaki y...,Ekonomi,2
2304,carlo ancelotti yönetimindeki napoli nde e gru...,Spor,0
8154,ekim’de vizyona girecek film için sosyal med...,Kültür-Sanat,3
9785,karadenizde yer alan iki adadan biri olan ve ...,Kültür-Sanat,3
2272,teknik direktör arayışlarında sona gelen trabz...,Spor,0
9632,engelsiz filmler festivalinin bir parçası olar...,Kültür-Sanat,3
9951,sinemada ‘cinsellik’ meselesine yaklaşmak ayrı...,Kültür-Sanat,3
3315,sahipleri uzun zamandır bataryalarının mümkün ...,Teknoloji,1
9207,oy i̇stanbul’un ilk prodüksiyonu olan “ ı”nın ...,Kültür-Sanat,3
8209,devlet opera ve balesinin dob uluslararası ala...,Kültür-Sanat,3


In [ ]:
new_df[new_df['content'].str.startswith('Kamu kurumları')==True]

,content,topic


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, TrainingArguments, BertTokenizerFast, BertForSequenceClassification

tokenizer = BertTokenizerFast.from_pretrained("dbmdz/bert-base-turkish-cased", max_length=510)
model = BertForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-cased", num_labels=4, id2label=id2label, label2id=label2id)

#data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
#from transformers import AutoTokenizer, BertForMaskedLM

#model = BertForMaskedLM.from_pretrained("ytu-ce-cosmos/turkish-base-bert-uncased")
#tokenizer = AutoTokenizer.from_pretrained("ytu-ce-cosmos/turkish-base-bert-uncased")

In [ ]:
train_dataset = df.sample(frac=0.6, random_state=42)
test_dataset = df.sample(frac=0.2, random_state=42)
val_dataset = df.sample(frac=0.2, random_state=42)

train_texts = train_dataset["text"]
train_labels = train_dataset["label"]

test_texts = test_dataset["text"]
test_labels = test_dataset["label"]

val_texts = val_dataset["text"]
val_labels = val_dataset["label"]

In [ ]:
train_texts.reset_index(drop=True, inplace=True)
train_labels.reset_index(drop=True, inplace=True)
val_texts.reset_index(drop=True, inplace=True)
val_labels.reset_index(drop=True, inplace=True)
test_texts.reset_index(drop=True, inplace=True)
test_labels.reset_index(drop=True, inplace=True)

In [ ]:
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True)

In [ ]:
len(test_encodings[2])

512

In [ ]:
import torch
from torch.utils.data import Dataset
class DataLoader(Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels
  def label_to_numeric(self, label):
      # Example mapping for categorical labels
      label_map = {'Spor': 0, 'Teknoloji': 1, 'Ekonomi': 2, 'Kültür-Sanat': 3}  # Adjust based on your actual labels
      return label_map[label]
  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    label = self.labels[idx]
    if isinstance(label, str):
        label = self.label_to_numeric(label)
    item['labels'] = torch.tensor(label)
    return item


  def __len__(self):
    return len(self.labels)

In [ ]:
train_dataloader = DataLoader(train_encodings, train_labels)
test_dataloader = DataLoader(test_encodings, test_labels)
val_dataloader = DataLoader(val_encodings, val_labels)

In [ ]:
len(val_encodings[9])

512

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy="steps",
    logging_steps=100,
    evaluation_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    fp16=True,
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import numpy as np
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataloader,
    eval_dataset=val_dataloader,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.809500,0.262193,0.931000,0.931604,0.942592,0.930188
200,0.170100,0.114987,0.972500,0.972323,0.972385,0.972357
300,0.142800,0.120953,0.968000,0.967729,0.970087,0.967600
400,0.160300,0.046990,0.986000,0.985944,0.985974,0.985970
500,0.080000,0.052010,0.985500,0.985418,0.985541,0.985506
600,0.062100,0.032836,0.991500,0.991464,0.991574,0.991453
700,0.069300,0.016037,0.995500,0.995473,0.995489,0.995467
800,0.043600,0.013978,0.996500,0.996487,0.996495,0.996495
900,0.029200,0.007930,0.998500,0.998488,0.998496,0.998484
1000,0.025100,0.006770,0.998500,0.998496,0.998496,0.998500


TrainOutput(global_step=1125, training_loss=0.14258359718322755, metrics={'train_runtime': 382.5049, 'train_samples_per_second': 47.058, 'train_steps_per_second': 2.941, 'total_flos': 4736084041728000.0, 'train_loss': 0.14258359718322755, 'epoch': 3.0})

In [ ]:
from transformers import pipeline
clf = pipeline("text-classification", model=model, tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
test_dataset.reset_index(drop=True, inplace=True)

In [ ]:
len(test_encodings)

3

 {'Spor': 0, 'Teknoloji': 1, 'Ekonomi': 2, 'Kültür-Sanat': 3}

In [ ]:
i = 0
score = 0
data_len = 0
for i in range(0, 2000):
  if len(val_texts[i])<=512:
    data_len = data_len + 1
    answer = clf(val_texts[i])
    print(f"{answer[0]['label']} : {val_labels[i]}")
    if str(val_labels[i]) == answer[0]["label"]:
      score = score + 1
    else:
      continue
  i = i + 1
print(data_len)
print(f"Accuracy: %{score*100/data_len}")

Ekonomi : Ekonomi
Ekonomi : Ekonomi
Teknoloji : Teknoloji
Kültür-Sanat : Kültür-Sanat
Kültür-Sanat : Kültür-Sanat
Ekonomi : Ekonomi
Spor : Spor
Kültür-Sanat : Kültür-Sanat
Spor : Spor
Ekonomi : Ekonomi
Kültür-Sanat : Kültür-Sanat
Spor : Spor
Spor : Spor
Spor : Spor
Spor : Spor
Ekonomi : Ekonomi
Kültür-Sanat : Kültür-Sanat
Ekonomi : Ekonomi
Spor : Spor
Teknoloji : Teknoloji
Spor : Spor
Spor : Spor
Kültür-Sanat : Kültür-Sanat
Kültür-Sanat : Kültür-Sanat
Ekonomi : Ekonomi
Spor : Spor
Teknoloji : Teknoloji
Spor : Spor
Ekonomi : Ekonomi
Kültür-Sanat : Kültür-Sanat
Kültür-Sanat : Kültür-Sanat
Spor : Spor
Spor : Spor
Spor : Spor
Ekonomi : Ekonomi
Spor : Spor
Spor : Spor
Spor : Spor
Spor : Spor
Kültür-Sanat : Kültür-Sanat
Ekonomi : Ekonomi
Kültür-Sanat : Kültür-Sanat
Teknoloji : Teknoloji
Ekonomi : Ekonomi
Spor : Spor
Ekonomi : Ekonomi
Kültür-Sanat : Kültür-Sanat
Spor : Spor
Spor : Spor
Ekonomi : Ekonomi
Ekonomi : Ekonomi
Spor : Spor
Kültür-Sanat : Kültür-Sanat
Kültür-Sanat : Kültür-Sanat
Ekon

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("bertTurkishModel")
tokenizer.push_to_hub("bertTurkishModel")

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mtolgakbabai16tr/bertTurkishModel/commit/3080f3b9ef5ddae26d0eeafa508c3ccec2adc509', commit_message='Upload tokenizer', commit_description='', oid='3080f3b9ef5ddae26d0eeafa508c3ccec2adc509', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
answer = clf("TL USD")
print(answer)

[{'label': 'Ekonomi', 'score': 0.9894730448722839}]


In [ ]:
answer = clf("Picasso gibisi gelmedi bu zamana kadar")
print(answer)

[{'label': 'Kültür-Sanat', 'score': 0.9930016398429871}]


In [ ]:
answer = clf("Apple hisseleri son zamanlarda yükselişte")
print(answer)

[{'label': 'Teknoloji', 'score': 0.9964046478271484}]


In [ ]:
answer = clf("Apple'ın son güncellemesi çok etkili oldu")
print(answer)

[{'label': 'Teknoloji', 'score': 0.9991171956062317}]


In [ ]:
answer = clf("Millilerimizi 2026 Dünya Kupasında görmek için sabırsızlanıyorum")
print(answer)

[{'label': 'Spor', 'score': 0.9991816878318787}]


In [ ]:
answer = clf("Djokovic o son sette naptı be abi")
print(answer)

[{'label': 'Spor', 'score': 0.9995526671409607}]


In [ ]:
model.save_pretrained("m-tolgakbaba/bertTurskishmodel")
tokenizer.save_pretrained("m-tolgakbaba/bertTurskishmodel")

('m-tolgakbaba/bertTurskishmodel/tokenizer_config.json',
 'm-tolgakbaba/bertTurskishmodel/special_tokens_map.json',
 'm-tolgakbaba/bertTurskishmodel/vocab.txt',
 'm-tolgakbaba/bertTurskishmodel/added_tokens.json',
 'm-tolgakbaba/bertTurskishmodel/tokenizer.json')